# Tutorial on PuLP

We will provide a brief tutorial on the PuLP package - a simple python based package that allows us to formulate and solve linear (and integer) programs. 


We import the package as follows:

In [ ]:
from pulp import *

**Step 1:** Creating a problem data-structure.

The 'name' provided is simply a descriptive name whereas the second argument specifies whether we seek to minimize or maximize an objective function.

In [ ]:
problem = LpProblem('name', LpMinimize) # Use LpMaximize if you are maximizing an objective function

**Step 2:** Create decision variables. 

We will declare a bunch of decision variables.
Let us create a variable named `x_1`.

In [ ]:
x_1 = LpVariable('x_1')

We now create variable `x_2`, `x_3`, `x_4` with lower bounds 0. In other words, as we create these variables, we also add the constraints `x_2 >= 0, x_3 >= 0 ` and `x_4 >= 0` to our problem.

In [ ]:
x_2 = LpVariable('x_2', 0)
x_3 = LpVariable('x_3', 0)
x_4 = LpVariable('x_4', 0)

We now create a variable `x_5` with lower bound -1 and upper bound +1. In other words, we would like to create `x_5` and at the same time add the constraints `x_5 >= -1 `, `x_5 <= 1` to the problem.

In [ ]:
x_5 = LpVariable('x_5', -1, 1)

**Step 3:** Set the objective function.

We would like our objective function to be `x_1 + x_2 - x_3 + 2 * x_4 - 3 * x_5`. Note that we already specified that we are minimizing it when we created the problem.

In [ ]:
problem += (x_1 + x_2 - x_3 + 2 * x_4 - 3 * x_5)

**Step 6:** We would like to add constraints

Constraint # 1:  ` x_1 - 2 * x_2 + x_3 <= 5 `
Constraint # 2:  ` 2 x_2 - x_4 + x_5 <= 7 `
Constraint # 3: ` x_1 - x_5 + 2 * x_4 <= 8`

In [ ]:
problem += (x_1 - 2 * x_2 + x_3 <= 5)
problem += (2* x_2 - x_4 + x_5  <= 7)
problem += (x_1 - x_5 + 2 * x_4 <= 8)

**Step 5:** Solve the problem and extract the solution.

In [ ]:
problem.solve() # solve the problem

In [ ]:
print('Status is :', LpStatus[problem.status])

**Step 6:** Check the status and do the appropriate things.

In [ ]:
if problem.status == constants.LpStatusOptimal:
    print('Optimal Solution Found!!')
    # Extract the values of the decision variables.
    v_1 = x_1.varValue
    v_2 = x_2.varValue
    v_3 = x_3.varValue
    v_4 = x_4.varValue
    v_5 = x_5.varValue
    print('x_1 = {v_1}, x_2 = {v_2}, x_3 = {v_3}, x_4 = {v_4}, x_5={v_5}')
elif problem.status == constants.LpStatusUnbounded:
    print('Unbounded solution -- need more constraints')
elif problem.status == constants.LpStatusInfeasible:
    print('Problem has no feasible solution')
else: 
    print('Problem has an undefined status -- something went wrong.')

Ooops: our problem is unbounded. We need more constraints to make it bounded. Let's add constraints

`x_1 >= -5` and  `x_3 <= 15`. We could have added these as bounds when we created the variables `x_1, x_2` but we can also add them as constraints.

In [ ]:
problem2 = LpProblem('name', LpMinimize) # Use LpMaximize if you are maximizing an objective function
x_1 = LpVariable('x_1')
x_2 = LpVariable('x_2', 0)
x_3 = LpVariable('x_3', 0)
x_4 = LpVariable('x_4', 0)
x_5 = LpVariable('x_5', -1, 1)
problem2 += (x_1 + x_2 - x_3 + 2*x_4 - 3 * x_5)
problem2 += (x_1 - 2 * x_2 + x_3 <= 5)
problem2 += (2* x_2 - x_4 + x_5  <= 7)
problem2 += (x_1 - x_5 + 2 * x_4 <= 8)
problem2 += x_1 >= -5
problem2 += x_3 <= 15


Let's make a nice function to solve our problem and extract the solution as a list.

In [ ]:
def solve_and_extract_solution(problem, list_of_vars):
    ## Solve and check status again
    problem.solve() # solve the problem
    print('Status is :', LpStatus[problem.status])
    if problem.status == constants.LpStatusOptimal:
        print('Optimal Solution Found!!')
        sols = [x.varValue for x in list_of_vars]
        ## Alternatively you could write a for loop 
        ## sols = []
        ## for x in list_of_vars:
        ##    sols.append(x.varValue)
        print('Solution:',sols)
        print('Objective Value:', problem.objective.value())
        return sols
    elif problem.status == constants.LpStatusUnbounded:
        print('Unbounded solution -- need more constraints')
        return None
    elif problem.status == constants.LpStatusInfeasible:
        print('Problem has no feasible solution')
        return None
    else: 
        print('Problem has an undefined status -- something went wrong.')
        return None

In [ ]:
solve_and_extract_solution(problem2, [x_1, x_2, x_3, x_4, x_5])

Often, we do not have the LP in the form above. We instead have some data from which we need to formulate the problem (see the notes on network flow problem) for an example. Let's solve some examples. 

Consider the LP of the form 

$$ \begin{array}{lrl}
\max& x_1 + x_2 + \cdots + x_n \\ 
\mathsf{s.t.}& x_{i} - x_{i+1} & \leq c_{i}, \ i = 1, \ldots, n-1 \\ 
& x_j & \leq d_j, & j = 1, \ldots, n \\ 
& x_j & \geq 0 & j = 1, \ldots, n \\ 
\end{array}$$

You are given the value of $n \geq 1$ and a lists $[d_1, \ldots, d_n ]$ of upper bounds and 
$[c_1, \ldots, c_{n-1}]$. 

Let's formulate and solve this problem.

In [ ]:
def formulate_and_solve(c_list, d_list):
    n = len(d_list)
    assert len(c_list) == n-1
    prob = LpProblem('special_problem', LpMaximize) # set it to maximize the objective
    # 1. Create Decision Variables
    ## option 1
    ## list_of_vars = []
    ## for i in range(n):
    ##     new_var = LpVariable(f'x_{i+1}', 0, d_list[i]) # create a variable named x_{i} with lower bound 0 
    ##     list_of_vars.append(new_var)
    ## more elegant one liner in python
    list_of_vars = [LpVariable(f'x_{j+1}', 0, d_list[j]) for j in range(n) ]
    # Note that the constraints x_j >= 0 and x_j <= d_j are taken care of already.
    print(f'Created list of variables: {list_of_vars}')
    # 2. Set the objective
    ## option 1
    ## e = 0
    ## for x in list_of_vars:
    ##     e = e + x
    ## prob += e
    ## More elegant option 2
    prob += lpSum(list_of_vars) ## sum up all the elements in list to create an expression
    # 3. Add constraints
    for i in range(n-1):
        prob += list_of_vars[i] - list_of_vars[i+1] <= c_list[i] # x_i - x_{i+1} <= c_i
    
    # 4. Solve and extract solution
    sol_list = solve_and_extract_solution(prob, list_of_vars)
    return sol_list 

In [ ]:
formulate_and_solve([1, 4, 6, 8, 0], [5, 1, 2, 3, 2, 6])